# Linear Regression: Diabetes & Blood Pressures

In [1]:
from data.create_data import *
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.formula.api as smf

%matplotlib inline

In [2]:
data = read_frmgham()
clean_data = data.dropna(subset=['diabetes', 'sysbp', 'diabp'])
diabetes = clean_data.diabetes
sysbp = clean_data.sysbp
diabp = clean_data.diabp

### Research Question
  1. Is there a significant association in being diabetic and blood pressures?
  
#### Diabetic
A patient is diabetic if they have a casual serum glucose level of > 200 mg/dL. 

The feature `diabetes` is a dichotomous/binary predictor, representing whether the patient is diabetic (**`1`**) or not (**`0`**).

# Simple Linear Regression (*with Categorical Predictors*)

Create a linear model to predict the quantitative response of `sysbp` as well as `diabp` using the predictor/feature `cigpday`.  
**`y = β`<sub>0</sub> + `β`<sub>1</sub>`x`<sub>1</sub>**

#### Method: Ordinary Least Squares (Linear Least Squares Fit)
Identify a linear regression model that minimizes the sum of squares of the residuals. In otherwords, find the least squares fit.

## Hypothesis Tests

### Response: Systolic Blood Pressure
Identify if there's a linear relationship between `diabetes` and `sysbp`. If so, is the apparent slope due to chance?

#### Variables:
  * Explanatory/Independent Variable (Feature): **`diabetes`** (*binary*)
     * **`1`**: diabetic
     * **`0`**: non-diabetic
  * Dependent Variable (Response): **`sysbp`** 

1) Hypothesis
  * **H<sub>0</sub>**: There is no significant relationship between being diabetic and systolic blood pressure (β<sub>1</sub>=0).
  * **H<sub>A</sub>**: There is a significant relationship between between being diabetic and systolic blood pressure (β<sub>1</sub> ≠ 0).
  
2) Statistical Test
Compute the test statistic & p-value
  * test statistic: **slope**(β<sub>1</sub> = β<sub>diabetes</sub>)

In [3]:
model = smf.ols(formula="sysbp ~ C(diabetes)", data=clean_data)
results = model.fit()

inter = results.params['Intercept']
slope = results.params[1]
print "intercept = %.3f" % inter
print "slope = %.3f" % slope

intercept = 135.603
slope = 15.814


#### Model Coefficients
  * β<sub>0</sub> (intercept) = 135.6
  * β<sub>1</sub> (slope) = 15.814

The test-statistic (slope) is 15.814. The slope coefficient when used for categorical predictors is the *difference in the response between `0` and `1`.

From the linear regression the model, the equation to predict `sysbp` using the predictor variable `diabetes` is:  

`y = 15.8 x + 137.9`

#### p-value

In [4]:
pval = results.pvalues[1]
pval

2.0135367951627243e-55

#### 95% Confidence Interval

In [5]:
list(results.conf_int().iloc[1,:])

[13.847545107562567, 17.779874847856526]

#### Summary Statistics

In [6]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sysbp   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     248.6
Date:                Thu, 16 Mar 2017   Prob (F-statistic):           2.01e-55
Time:                        18:20:47   Log-Likelihood:                -52729.
No. Observations:               11627   AIC:                         1.055e+05
Df Residuals:                   11625   BIC:                         1.055e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept          135.6033      0.214    633.198      0.000       135.183   136.023
C(diabetes)[T.1]    15.8137      1.003     15.765      0.000        13.848    17.780
==============================================================================
Omnibus:                     1528.673   Durbin-Watson:                   1.214
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2481.226
Skew:                           0.910   Prob(JB):                         0.00
Kurtosis:                       4.345   Cond. No.                         4.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3) Results
#### Decide to Accept or Reject the null H<sub>0</sub>
Since the p-value < 0.05 and the confidence interval does not include 0, we reject the null hypothesis in favor of the alternative. There is substantial evidence against the null hypothesis.

#### Conclusion
Based on our sample, there is a statistically significant association between being diabetic and an individual's systolic blood pressure.

The estimated difference between *diabetics* and *non-diabetics* is 15.81 mmHg (β<sub>1</sub>).
  * Being a patient with diabetes (**`1`**) is associated with an average *increase* of β<sub>1</sub>.
  * Being a non-diabetic patient (**`0`**) is associated with an average *decrease* of β<sub>1</sub>.

### Goodness of Fit
Determine how well does the model fit the data.

#### RMSE
The root mean squared error is the standard deviation of the residuals. It is used to quantify the strength of relationship between `diabetes` and `sysbp`.

It measures the absolute fit of the model to the data (how close to the observed data points are to the model's predicted values).

In [7]:
rmse = results.resid.std()
print "RMSE of sysbp (residual) = %.3f mmHg" % rmse

RMSE of sysbp (residual) = 22.559 mmHg


In [8]:
rmse2 = clean_data['sysbp'].std()
print "RMSE of sysbp (y) = %.3f mmHg" % rmse2

RMSE of sysbp (y) = 22.799 mmHg


In [9]:
print "[%3f, %3f]" % (sysbp.min(), sysbp.max())

[83.500000, 295.000000]


##### Conclusion
The RMSE of the model is approximately 22.6 mmHg. It's relatively small in comparison to the range of the dependent variable. Hence, the low the RMSE value suggests a good fit.

Comparing the standard deviation of residuals and systolic blood pressure (ys), it appears knowing if the patient is diabetic does not drastically improve the model's prediction.
  * Difference of only .24 mmHg

#### Coefficient of Determination (R<sup>2</sup>)
The coefficient of determination quantifies the goodness of fit of the linear model to the data, demonstrating the model's predictive power.

The range of the value is [0, 1].

In [10]:
results.rsquared

0.020933107325236788

##### Conclusion
The R<sup>2</sup> value is 0.021, which indicates that the model, given  the information of whether the patient is diabetic, only predicts about 1.1% of variance in systolic blood pressure. 

The small value suggests that the proposed linear regression model does not improve prediction and has low predictive power. 

### Response: Diastolic Blood Pressure
Identify if there's a linear relationship between `diabetes` and `diabp`. If so, is the apparent slope due to chance?

#### Variables:
  * Explanatory/Independent Variable (Feature): **`diabetes`** (*binary*)
     * **`1`**: diabetic
     * **`0`**: non-diabetic
  * Dependent Variable (Response): **`diabp`** 

1) Hypothesis
  * **H<sub>0</sub>**: There is no significant relationship between being diabetic and diastolic blood pressure (β<sub>1</sub>=0).
  * **H<sub>A</sub>**: There is a significant relationship between between being diabetic and diastolic blood pressure (β<sub>1</sub> ≠ 0).
  
2) Statistical Test
Compute the test statistic & p-value
  * test statistic: **slope**(β<sub>1</sub> = β<sub>diabetes</sub>)

In [11]:
model_dia = smf.ols(formula="diabp ~ C(diabetes)", data=clean_data)
results_dia = model_dia.fit()

inter_dia = results_dia.params['Intercept']
slope_dia = results_dia.params[1]
print "intercept = %.3f" % inter_dia
print "slope = %.3f" % slope_dia

intercept = 82.957
slope = 1.778


#### Model Coefficients
  * β<sub>0</sub> (intercept) = 82.96
  * β<sub>1</sub> = 1.78

The test-statistic (β<sub>1</sub>) is 1.78. The slope coefficient when used for categorical predictors is the *difference in the response between `0` and `1`.

From the linear regression the model, the equation to predict `sysbp` using the predictor variable `diabetes` is:  

`y = 1.78 x + 82.96`

#### p-value

In [12]:
pval = results_dia.pvalues[1]
pval

0.00060230467592501833

#### 95% Confidence Interval

In [13]:
list(results_dia.conf_int().iloc[1,:])

[0.76245016001020582, 2.7939611427216327]

#### Summary Statistics

In [14]:
results_dia.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  diabp   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     11.78
Date:                Thu, 16 Mar 2017   Prob (F-statistic):           0.000602
Time:                        18:20:47   Log-Likelihood:                -45050.
No. Observations:               11627   AIC:                         9.010e+04
Df Residuals:                   11625   BIC:                         9.012e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           82.9567      0.111    749.810      0.000        82.740    83.174
C(diabetes)[T.1]     1.7782      0.518      3.432      0.001         0.762     2.794
==============================================================================
Omnibus:                      701.505   Durbin-Watson:                   1.290
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              967.773
Skew:                           0.545   Prob(JB):                    7.09e-211
Kurtosis:                       3.900   Cond. No.                         4.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3) Results
#### Decide to Accept or Reject the null H<sub>0</sub>
Since the p-value < 0.05 and the confidence interval does not include 0, we reject the null hypothesis in favor of the alternative. There is substantial evidence against the null hypothesis.

#### Conclusion
Based on our sample, there is a statistically significant association between being diabetic and an individual's diastolic blood pressure.

The estimated difference between *diabetics* and *non-diabetics* is 1.78 mmHg (β<sub>1</sub>).
  * Being a patient with diabetes (**`1`**) is associated with an average *increase* of β<sub>1</sub>.
  * Being a non-diabetic patient (**`0`**) is associated with an average *decrease* of β<sub>1</sub>.

### Goodness of Fit

#### RMSE

In [15]:
rmse = results_dia.resid.std()
print "RMSE of diabp (residual) = %.3f mmHg" % rmse

rmse2 = clean_data['diabp'].std()
print "RMSE of diabp (y) = %.3f mmHg" % rmse2

print "[%3f, %3f]" % (diabp.min(), diabp.max())

RMSE of diabp (residual) = 11.654 mmHg
RMSE of diabp (y) = 11.660 mmHg
[30.000000, 150.000000]


##### Conclusion
The RMSE of the model is approximately 11.65 mmHg. It's relatively small in comparison to the range of the dependent variable. Hence, the low the RMSE value suggests a good fit.

Comparing the standard deviation of residuals and diastolic blood pressure (ys), it appears knowing if the patient is diabetic does not drastically improve the model's prediction.
  * Difference of only .06 mmHg

#### Coefficient of Determination (R<sup>2</sup>)

In [16]:
results_dia.rsquared

0.001011904640854655

##### Conclusion
The R<sup>2</sup> value is 0.001. The model, given the information of whether the patient is diabetic, only predicts about 0.1% of variance in diastolic blood pressure. 

The small value suggests that the proposed linear regression model does not improve prediction and has low predictive power. 